# Decoding continuous eye movements from cortical spiking activity
### Kendra K. Noneman and J. Patrick Mayo

In [1]:
DATA_FOLDER =  '/Users/kendranoneman/Projects/mayo/NeuralDecoding/runs2/'
FIG_FOLDER  =  '/Users/kendranoneman/Figures/neural-decoding/final_manuscript'

saveFigs    =  1 # 1 = save figs, 0 = don't save figs

## Packages and functions

In [2]:
# Packages and toolboxes
import numpy as np
import pandas as pd
import os
import sys
import pickle
import cmath
import math

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline

from scipy import io, stats
from scipy.stats import mannwhitneyu, normaltest, zscore, wilcoxon, ttest_rel

# Colors for each decoding model
cols = ['#dc6d3f','#f1c232','#317c32','#20a693','#358eb9','#3127d7','#a624d7','#d51f89']
cols = [mcolors.to_rgb(col) for col in cols]

grays = ['#f4d3c5','#faecc1','#c1d7c1','#bce4de','#c2ddea','#c1bef3','#e4bdf3','#f2bbdb']
grays = [mcolors.to_rgb(col) for col in grays]

pal_cols = ['#B3AA00','#74B300','#00B315','#00B377','#008CB3','#002DB3','#3C00B3','#8C00B3']
pals = [sns.light_palette(col, as_cmap=True, n_colors=5) for col in cols]

In [3]:
example_data = io.loadmat('/Users/kendranoneman/Projects/mayo/NeuralDecoding/datasets/vars/vars-pa29dir4A-pre500-post300.mat')
out_times    =  np.squeeze(example_data['vels_times']) # times at which velocities were recorded
sess_size = len(out_times)

In [4]:
# Functions for concatenating pickle files and calculating evaluation metrics
def get_outputs_wTraces(data_folder,conditions,column_names):
    """ 
    Function to get results for each decoder run, concatenated across models, repeats, etc...

    Parameters
    ----------
    data_folder  - path to the folder where data is located 
    conditions   - array of strings, containing labels for conditions you wish to include in data extraction
    column_names - names of columns for data you are pulling out, this varied a little across analyses

    Returns
    -------
    df            - pandas dataframe containing all of the evaluation metrics and labels for each decoder run
    conds_all     - labels for each observation in the test set, including trial conditions and markers
    yTest_all     - actual/recorded eye traces from test set
    yTestPred_all - predicted/decoded eye traces from test set 
    """
    results_all,conds_all,yTest_all,yTestPred_all = [],[],[],[]
    direcs = sorted(os.listdir(data_folder))
    current_direcs = [d for d in direcs if all(c in d for c in conditions)]
    for direc in current_direcs:
        files = sorted(os.listdir(os.path.join(data_folder, direc)))
        for file in files:
            file_path = os.path.join(data_folder, direc, file)
            if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
                if file.endswith('.pickle') and ('repeat000.' in file or 'repeat00.' in file):
                    with open(file_path, 'rb') as f:
                        results,conds,y_test,y_test_predicted = pickle.load(f)
                        conds_all.append(conds)
                        results_all.append(results)
                        yTest_all.append(y_test)
                        yTestPred_all.append(y_test_predicted)
                elif file.endswith('.pickle'):
                    with open(file_path, 'rb') as f:
                        results = pickle.load(f)
                        results_all.append(results[0])

    df = pd.DataFrame(results_all,columns=column_names)
    df['R2_x']   = df['R2'].apply(lambda x: x[0])
    df['R2_y']   = df['R2'].apply(lambda x: x[1])
    df['R2_mn']  = df['R2'].apply(lambda x: x.mean())
    df['rho_mn']  = df['rho'].apply(lambda x: x.mean())
    df['R2sh_mn']  = df['R2_shuf'].apply(lambda x: x.mean())
    df['R2tr_mn']  = df['R2_train'].apply(lambda x: x.mean())
    
    return df,conds_all,yTest_all,yTestPred_all

def get_outputs(data_folder, conditions, column_names):
    """ 
    Function to get results for each decoder run, concatenated across models, repeats, etc...

    Parameters
    ----------
    data_folder  - path to the folder where data is located 
    conditions   - array of strings, containing labels for conditions you wish to include in data extraction
    column_names - names of columns for data you are pulling out, this varied a little across analyses

    Returns
    -------
    df            - pandas dataframe containing all of the evaluation metrics and labels for each decoder run
    yTestPred_all - predicted/decoded eye traces from test set 
    """
    results_all, y_pred_all = [], []

    # Get sorted list of directories in data_folder
    direcs = sorted(os.listdir(data_folder))

    # Filter directories based on conditions
    current_direcs = [d for d in direcs if all(c in d for c in conditions)]

    # Iterate over filtered directories
    for direc in current_direcs:
        files = sorted(os.listdir(os.path.join(data_folder, direc)))
        for file in files:
            file_path = os.path.join(data_folder, direc, file)
            # Check if file exists and is non-empty
            if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
                with open(file_path, 'rb') as f:
                    # Load data from file
                    results, y_pred = pickle.load(f)
                    # Append to lists
                    results_all.append(results)
                    y_pred_all.append(y_pred)

    # Create DataFrame from results_all list
    df = pd.DataFrame(results_all, columns=column_names)

    return df, y_pred_all

def get_actualTraces(data_folder, df):

    
    
    results_all, y_pred_all = [], []

    # Get sorted list of directories in data_folder
    direcs = sorted(os.listdir(data_folder))

    # Filter directories based on conditions
    current_direcs = [d for d in direcs if all(c in d for c in conditions)]

    # Iterate over filtered directories
    for direc in current_direcs:
        files = sorted(os.listdir(os.path.join(data_folder, direc)))
        for file in files:
            file_path = os.path.join(data_folder, direc, file)
            # Check if file exists and is non-empty
            if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
                with open(file_path, 'rb') as f:
                    # Load data from file
                    results, y_pred = pickle.load(f)
                    # Append to lists
                    results_all.append(results)
                    y_pred_all.append(y_pred)

    # Create DataFrame from results_all list
    df = pd.DataFrame(results_all, columns=column_names)

    return df, y_pred_all

def get_R2(y_test,y_test_pred):
    """ 
    Function to get R2

    Parameters
    ----------
    y_test      - the true outputs (a matrix of size number of examples x number of outputs)
    y_test_pred - the predicted outputs (a matrix of size number of examples x number of outputs)

    Returns
    -------
    R2_array - An array of R2s for each output
    """
    y_mean = np.mean(y_test, axis=0)
    numer = np.sum((y_test_pred - y_test) ** 2, axis=0)
    denom = np.sum((y_test - y_mean) ** 2, axis=0)
    
    R2 = 1 - numer / denom #Append R2 of this output to the list

    return R2  #Return an array of R2s

######## BIN_SPIKES ########
def bin_spikes(spike_times,dt,wdw_start,wdw_end):
    """ 
    Function that puts spikes into bins

    Parameters
    ----------
    spike_times: an array of arrays
        an array of neurons. within each neuron's array is an array containing all the spike times of that neuron
    dt: number (any format)
        size of time bins
    wdw_start: number (any format)
        the start time for putting spikes in bins
    wdw_end: number (any format)
        the end time for putting spikes in bins

    Returns
    -------
    neural_data: a matrix of size "number of time bins" x "number of neurons"
        the number of spikes in each time bin for each neuron
    """
    edges=np.arange(wdw_start,wdw_end,dt) #Get edges of time bins
    num_bins=edges.shape[0]-1 #Number of bins
    num_neurons=spike_times.shape[0] #Number of neurons
    neural_data=np.empty([num_bins,num_neurons]) #Initialize array for binned neural data
    #Count number of spikes in each bin for each neuron, and put in array
    for i in range(num_neurons):
        neural_data[:,i]=np.histogram(spike_times[i],edges)[0]
    
    return neural_data

######## BIN_OUTPUT #######
def bin_output(outputs,output_times,dt,wdw_start,wdw_end,downsample_factor=1):
    """ 
    Function that puts outputs into bins

    Parameters
    ----------
    outputs: matrix of size "number of times the output was recorded" x "number of features in the output"
        each entry in the matrix is the value of the output feature
    output_times: a vector of size "number of times the output was recorded"
        each entry has the time the output was recorded
    dt: number (any format)
        size of time bins
    wdw_start: number (any format)
        the start time for binning the outputs
    wdw_end: number (any format)
        the end time for binning the outputs
    downsample_factor: integer, optional, default=1
        how much to downsample the outputs prior to binning
        larger values will increase speed, but decrease precision

    Returns
    -------
    outputs_binned: matrix of size "number of time bins" x "number of features in the output"
        the average value of each output feature in every time bin
    """

    ###Downsample output###
    #We just take 1 out of every "downsample_factor" values#
    if downsample_factor!=1: #Don't downsample if downsample_factor=1
        downsample_idxs=np.arange(0,output_times.shape[0],downsample_factor) #Get the idxs of values we are going to include after downsampling
        outputs=outputs[downsample_idxs,:] #Get the downsampled outputs
        output_times=output_times[downsample_idxs] #Get the downsampled output times

    ###Put outputs into bins###
    edges=np.arange(wdw_start,wdw_end,dt) #Get edges of time bins
    num_bins=edges.shape[0]-1
    output_dim=outputs.shape[1]
    outputs_binned=np.empty([num_bins,output_dim])
    #Loop through bins, and get the mean outputs in those bins
    for i in range(num_bins):
        idxs=(np.squeeze(output_times)>=edges[i]) & (np.squeeze(output_times)<edges[i+1])
        outputs_binned[i,:] = outputs[idxs,:].mean(axis=0)

    return outputs_binned,edges

def get_exTrialLabels(df,conds,best_fold,dto,y_test,y_test_predicted,mdl):
    c = conds[df[(df['outer_fold'] == best_fold) & (df['output_binWidth'] == dto) & (df['model'] == mdl)].index.values[0]]
    y = y_test[df[(df['outer_fold'] == best_fold) & (df['output_binWidth'] == dto) & (df['model'] == mdl)].index.values[0]]
    yp = y_test_predicted[df[(df['outer_fold'] == best_fold) & (df['output_binWidth'] == dto) & (df['model'] == mdl)].index.values[0]]
    
    trl_nums, indices = np.unique(c[:,0], return_index=True)
    trl_dirs = c[indices,3]
    
    R2_all = []
    for t in trl_nums:
        start_end = np.where(np.isin(c[:, 0],t))[0]

        X_true = y[start_end,0]
        Y_true = y[start_end,1]

        X_pred = yp[start_end,0]
        Y_pred = yp[start_end,1]
        
        R2 = get_R2(np.concatenate((X_true.reshape(-1,1),Y_true.reshape(-1,1)),axis=1),np.concatenate((X_pred.reshape(-1,1),Y_pred.reshape(-1,1)),axis=1))
        R2_all.append(R2.mean())

    R2_all = np.array(R2_all)
    
    ex_trials = []
    for d in np.sort(np.unique(trl_dirs)):
        d_indices = np.where(trl_dirs == d)[0]
        d_R2 = R2_all[d_indices]
        d_trl = trl_nums[d_indices]
        d_dir = trl_dirs[d_indices]

        sorted_indices = np.argsort(d_R2)
        
        ex_trials.extend(d_trl[sorted_indices[-1:]].astype(int))
    
    return ex_trials

def get_eyeTraces(df,conds,fold,dto,dti,wi,trls,y_test,y_test_predicted,mdl):
    c = conds[df[(df['outer_fold'] == fold) & (df['output_binWidth'] == dto) & (df['input_binWidth'] == dti) & (df['input_timeWindow'] == wi) & (df['model'] == mdl)].index.values[0]]
    y = y_test[df[(df['outer_fold'] == fold) & (df['output_binWidth'] == dto) & (df['input_binWidth'] == dti) & (df['input_timeWindow'] == wi) & (df['model'] == mdl)].index.values[0]]
    yp = y_test_predicted[df[(df['outer_fold'] == fold) & (df['output_binWidth'] == dto) & (df['input_binWidth'] == dti) & (df['input_timeWindow'] == wi) & (df['model'] == mdl)].index.values[0]]
    
    inds = np.where(np.isin(c[:,0],trls))[0]
    
    return y[inds,:], yp[inds,:]

def get_dfVal_folds(df,dto,dti,wi,val):
    wf = (df.loc[(df['model']==0) & (df['output_binWidth']==dto) & (df['input_binWidth']==dti) & (df['input_timeWindow']==wi)][val]).values
    wc = (df.loc[(df['model']==1) & (df['output_binWidth']==dto) & (df['input_binWidth']==dti) & (df['input_timeWindow']==wi)][val]).values
    xgb = (df.loc[(df['model']==3) & (df['output_binWidth']==dto) & (df['input_binWidth']==dti) & (df['input_timeWindow']==wi)][val]).values
    svr = (df.loc[(df['model']==4) & (df['output_binWidth']==dto) & (df['input_binWidth']==dti) & (df['input_timeWindow']==wi)][val]).values
    dnn = (df.loc[(df['model']==5) & (df['output_binWidth']==dto) & (df['input_binWidth']==dti) & (df['input_timeWindow']==wi)][val]).values
    rnn = (df.loc[(df['model']==6) & (df['output_binWidth']==dto) & (df['input_binWidth']==dti) & (df['input_timeWindow']==wi)][val]).values
    gru = (df.loc[(df['model']==7) & (df['output_binWidth']==dto) & (df['input_binWidth']==dti) & (df['input_timeWindow']==wi)][val]).values
    lstm = (df.loc[(df['model']==8) & (df['output_binWidth']==dto) & (df['input_binWidth']==dti) & (df['input_timeWindow']==wi)][val]).values
    
    return wf,wc,xgb,svr,dnn,rnn,gru,lstm

def get_goodSessions(df,min_r2):
    sessions = []
    for i,s in enumerate(df['sess'].unique()):
        if s==39 or s==28 or s==23 or s==27:
            continue
        else:
            wf,wc,xgb,svr,dnn,rnn,gru,lstm = get_dfVal_folds(df.loc[(df['sess']==s)],50,50,400,'R2_mn')

            n=10
            means=([np.mean(wf),np.mean(wc),np.mean(xgb),np.mean(svr),np.mean(dnn),np.mean(rnn),np.mean(gru),np.mean(lstm)])    
            err=([np.std(wf)*np.sqrt(1./n+1./(n-1)),np.std(wc)*np.sqrt(1./n+1./(n-1)),np.std(xgb)*np.sqrt(1./n+1./(n-1)),np.std(svr)*np.sqrt(1./n+1./(n-1)),np.std(dnn)*np.sqrt(1./n+1./(n-1)),np.std(rnn)*np.sqrt(1./n+1./(n-1)),np.std(gru)*np.sqrt(1./n+1./(n-1)),np.std(lstm)*np.sqrt(1./n+1./(n-1))])

            if np.any(np.array(means) < min_r2):
                continue
            else:
                sessions.append(s)
                
    return sessions

def calc_evalMetrics(df,sess_size,minR2):
    df = df[df['R2_mn'] >= minR2]
    
    df['num_obs']      =  sess_size / df['output_binWidth'] # number of total observations
    df['num_feat'] =  df['input_timeWindow'] / df['input_binWidth'] # number of features
    
    # training time
    df['train_time']    =  df['train_time'] / 60   # convert time to sec
    
    # inference latency
    df['preProcessTime_corrected'] =  df['preProcess_time'] / df['num_obs']
    df['inference_time']           =  df['test_time'] + df['preProcessTime_corrected']
    
    # generalization gap
    df['robust_diff'] =  ((df['R2tr_mn'] - df['R2_mn']) / df['R2tr_mn']) * 100
    
    return df
    

## Figure 1. Decoding pipeline and example trials

In [7]:
column_names = ['line','sess','timePrePost','output_binWidth','downsample_factor','input_timeWindow','input_binWidth','numNeurons','nMT','nFEF','outer_fold','repeat','trainRatio','numObservations','output','model','validEvalMetric','prms','preProcess_time','train_time','test_time','R2_test']
df, y_pred = get_outputs(DATA_FOLDER,["r0001"],column_names) # position

df.head(2)

# Get trial numbers for "best" trial in each direction, will be used for all eye traces plots
#plot_trls = get_exTrialLabels(df0,conds0,0,50,yTest_all0,yTestPred_all0,5)
#print(plot_trls)

#df0.head(2)

,line,sess,timePrePost,output_binWidth,downsample_factor,input_timeWindow,input_binWidth,numNeurons,nMT,nFEF,...,trainRatio,numObservations,output,model,validEvalMetric,prms,preProcess_time,train_time,test_time,R2_test
0,0,17,0,50,1,400,50,49,24,25,...,1.0,28491,position,0,R2,{'nan': 0},1786.616246,1.138448,0.000026,"[0.49525688449061744, 0.22550321589827726]"
1,0,17,0,50,1,400,50,49,24,25,...,1.0,28483,position,0,R2,{'nan': 0},1786.616246,1.123332,0.000028,"[0.4886043156202281, 0.219353846334142]"


In [ ]:
####### (a) example traces for single trial, actual eye traces and predicted eye traces ###########

pipe_trl = 1861

# example recorded eye trace
fig, ax = plt.subplots(figsize=(4.5, 2.75))
fig.tight_layout()

y_true, y_pred = get_eyeTraces(df0,conds0,4,50,50,400,pipe_trl,yTest_all0,yTestPred_all0,8)
x = np.linspace(0,1*1600,y_true.shape[0])

ax.plot(x,y_true[:,0],linewidth=4,linestyle='dashed',color=(0.6,0.6,0.6))
ax.plot(x,y_true[:,1],linewidth=4,linestyle='solid',color=(0.6,0.6,0.6))

ax.axis('off')
ax.grid(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

fig.savefig(FIG_FOLDER+'/actual_eyeTrace.png', dpi=600, bbox_inches='tight')

# example recorded/decoded eye trace
fig, ax = plt.subplots(figsize=(5, 3))
fig.tight_layout()

y_true, y_pred = get_eyeTraces(df0,conds0,4,50,50,400,pipe_trl,yTest_all0,yTestPred_all0,8)
x = np.linspace(0,1*1600,y_true.shape[0])

ax.plot(x,y_true[:,0],linewidth=4,linestyle='dashed',color=(0.6,0.6,0.6))
ax.plot(x,y_true[:,1],linewidth=4,linestyle='solid',color=(0.6,0.6,0.6))

ax.plot(x,y_pred[:,0],linewidth=4,linestyle='dashed',color=cols[7])
ax.plot(x,y_pred[:,1],linewidth=4,linestyle='solid',color=cols[7])

ax.axis('off')
ax.grid(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

if saveFigs==1:
    fig.savefig(FIG_FOLDER+'/actualDecoded_eyeTrace.png', dpi=600, bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(7,3)) #6,5.5
plt.rcParams.update({'font.size': 10.5})
fig.tight_layout()

for i, ax in enumerate(axes.flatten()):
    y_true, y_pred = get_eyeTraces(df0,conds0,0,50,50,400,plot_trls[i],yTest_all0,yTestPred_all0,8)
    x = np.linspace(0-500,(1*1600)-500,y_true.shape[0])

    ax.plot(x,y_true[:,0],linewidth=2,linestyle='solid',color=(0.6,0.6,0.6),label='$x_{recorded}$')
    ax.plot(x,y_true[:,1],linewidth=2,linestyle='dashed',color=(0.6,0.6,0.6),label='$y_{recorded}$')

    ax.plot(x,y_pred[:,0],linewidth=2,linestyle='solid',color=cols[7],label='$x_{predicted}$')
    ax.plot(x,y_pred[:,1],linewidth=2,linestyle='dashed',color=cols[7],label='$y_{predicted}$')
    
    ax.axhline(0, color='black', linestyle='-',linewidth=0.5,zorder=0)  # Dashed line at y=0
    #plt.axvline(0, color='gray', linestyle='--')  # Dashed line at x=0

plt.subplots_adjust(wspace=0.15, hspace=0.05)
    
pos1 = axes[0,0].get_position()
axes[0,0].set_position(axes[0,1].get_position())
axes[0,1].set_position(pos1)

axes[0,0].set_ylim(-1.5,12.5)
axes[0,1].set_ylim(-14.5,14.5)
axes[1,0].set_ylim(-12.5,2.5)
axes[1,1].set_ylim(-14.5,14.5)

axes[0,0].set_xticks([])
axes[0,1].set_xticks([])
axes[1,0].set_xticks([-500,0,500,1000])
axes[1,0].set_xticks([-500,0,500,1000])

axes[0,1].legend(loc='lower center', bbox_to_anchor=(1.1, 0.95),
          fancybox=False, shadow=False, ncol=4, fontsize="11.5",frameon=False)

if saveFigs==1:
    fig.savefig(FIG_FOLDER+'/model_4traces_subplots.png', dpi=600, bbox_inches='tight')

In [ ]:
from matplotlib.collections import LineCollection

fig, ax = plt.subplots(figsize=(7.5, 3))
plt.rcParams.update({'font.size': 11.5})
fig.tight_layout()

# Generate example data (replace this with your actual data)
handles = []
labels = []
# Plotting lines starting from (0, 0) to the calculated endpoints
for i, angle_deg in enumerate([55, 145, 235, 325]):
    angle_rad = np.radians(angle_deg)
    line_length = 16
    endpoint = (line_length * np.cos(angle_rad), line_length * np.sin(angle_rad))
    
    line, = ax.plot([0, endpoint[0]], [0, endpoint[1]], color=(0.1, 0.1, 0.1), linestyle=':', linewidth=2, zorder=1)
    if i == 0:
        handles.append(line)
        labels.append('target')
        
for i in range(4):
    y_true, y_pred = get_eyeTraces(df0,conds0,0,50,50,400,plot_trls[i],yTest_all0,yTestPred_all0,5)
    
    # Plot the true eye traces
    true_line, = ax.plot(y_true[:, 0], y_true[:, 1], linewidth=2, linestyle='solid', color=(0.6, 0.6, 0.6), zorder=2)
    if i == 0:
        handles.append(true_line)
        labels.append('recorded eye')
    
    decoded_line, = ax.plot(y_pred[:, 0], y_pred[:, 1], linewidth=2, linestyle='solid', color=cols[7], zorder=3)
    if i == 0:
        handles.append(decoded_line)
        labels.append('predicted eye')

ax.axhline(0, color='black', linestyle='-',linewidth=0.5,zorder=0)  # Dashed line at y=0
ax.axvline(0, color='black', linestyle='-',linewidth=0.5,zorder=0)  # Dashed line at x=0

plt.gca().set_aspect('equal', adjustable='box')

# Create legend with custom handles and labels
ax.legend(handles=handles, labels=labels, loc='lower center', bbox_to_anchor=(0.5, 0.95),
          fancybox=False, shadow=False, ncol=3, fontsize="13", frameon=False)

ax.set_xlim([-17,17])
ax.set_xticks([-10,0,10])
ax.set_ylim([-17,17])
ax.set_yticks([-10,0,10])
ax.grid(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.gca().set_aspect('equal', adjustable='box')

if saveFigs==1:
    fig.savefig(FIG_FOLDER+'/model_4traces.png', dpi=600, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5.75,2.75)) #6,5.5
plt.rcParams.update({'font.size': 12})
fig.tight_layout()

wf,wc,xgb,svr,dnn,rnn,gru,lstm = get_dfVal_folds(df0,50,50,400,'R2_mn')

n=10
means=([np.mean(wf),np.mean(wc),np.mean(xgb),np.mean(svr),np.mean(dnn),np.mean(rnn),np.mean(gru),np.mean(lstm)])
err=([np.std(wf)*np.sqrt(1./n+1./(n-1)),np.std(wc)*np.sqrt(1./n+1./(n-1)),np.std(xgb)*np.sqrt(1./n+1./(n-1)),np.std(svr)*np.sqrt(1./n+1./(n-1)),np.std(dnn)*np.sqrt(1./n+1./(n-1)),np.std(rnn)*np.sqrt(1./n+1./(n-1)),np.std(gru)*np.sqrt(1./n+1./(n-1)),np.std(lstm)*np.sqrt(1./n+1./(n-1))])

ind = np.arange(len(err)) #X values for plotting
for pos, y, yerr, color in zip(ind, means, err, cols): #Loop through methods and plot error bars
    ax.errorbar(pos, y, yerr, lw=2.5, capsize=7.5, capthick=2, color=color)

    ax.set_xticklabels(['','wf','wc','xgb','svr','dnn','rnn','gru','lstm'], rotation=0)

    #Remove right and top boundaries, and make ticks inward
    ax.tick_params(direction='in',bottom=0)
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')

    #Plot individual R2 values for each fold as an 'x'
    scatter_x=np.reshape(np.transpose(np.ones((10,1))*range(8)),(80,1)) #Get x values for plotting (first 10 will have an x value of 0, second 10 will have an x value of 1, etc)
    scatter_y=np.concatenate((wf,wc,xgb,svr,dnn,rnn,gru,lstm),axis=0) #Y values for plotting
    colors_list=[] #Create a list of the colors that should be used when plotting each 'x'
    for i in scatter_x.astype(int).reshape((1,-1))[0]:
        colors_list.append(cols[i])
        
    ax.scatter(scatter_x,scatter_y,c=colors_list,marker='x',s=60,alpha=0.04)
 
if saveFigs==1:
    fig.savefig(FIG_FOLDER+'/r2_10folds.png', dpi=600, bbox_inches='tight')

## Figure 2. Detailed decoding schematic

In [ ]:
dt = 50
data = io.loadmat('/Users/kendranoneman/Projects/mayo/NeuralDecoding/datasets/vars/vars-pa29dir4A-pre500-post300.mat')

spike_times  =  data['spike_times'] # spike times of all neurons
pos          =  data['pos'] # x and y eye positions
vel         =  data['vels']
acc          =  data['acc']
out_times    =  data['vels_times'] # times at which velocities were recorded
out_times    =  np.squeeze(out_times)

conditions = data['contConditions']
print(np.unique(conditions[:,0]).shape)

spike_times  =  np.squeeze(spike_times)
for i in range(spike_times.shape[0]):
    spike_times[i]  =  np.squeeze(spike_times[i])
    
unts = pd.read_csv('/Users/kendranoneman/Projects/mayo/NeuralDecoding/datasets/units/units-pa29dir4A-pre500-post300.csv')
fef = unts[(unts['signiffl'] == 1) & (unts['BrainArea'] == 'FEF') & (unts['DI'] > 0.175)].index
mt = unts[(unts['signiffl'] == 1) & (unts['BrainArea'] == 'MT') & (unts['DI'] > 0.175)].index

print(fef.shape[0] + mt.shape[0])
    
##############################################################################################
#trial = [1861] #1858. 1861. 1879. 1882. 1883.

trls = [plot_trls[3]]
trls = [1858,1861]

spikes = []
for sublist in spike_times:
    s = sublist[np.logical_and(sublist>np.where(np.isin(conditions[:,0],trls))[0][0], sublist<np.where(np.isin(conditions[:,0],trls))[0][-1])]
    spikes.append(s)
    
fig, ax = plt.subplots(figsize=(10.2, 3))
fig.tight_layout()

ax.eventplot([spikes[i] for i in fef], colors='black', linelengths=0.5, linewidths=0.75, lineoffsets=np.arange(len([spikes[i] for i in fef])));
ax.eventplot([spikes[i] for i in mt], colors='black', linelengths=0.5, linewidths=0.75, lineoffsets=np.arange(len([spikes[i] for i in fef]),len([spikes[i] for i in fef])+len([spikes[i] for i in mt])));

ax.axis('off')
ax.grid(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

if saveFigs==1:
    fig.savefig(FIG_FOLDER+'/examp_spikes.png', dpi=600, bbox_inches='tight')

##############################################################
fig2a, ax = plt.subplots(figsize=(10.2, 2.25))
fig2a.tight_layout()

poses = pos[np.isin(conditions[:,0],trls)]
ttt = 4
print([poses[320*ttt:320*(ttt+1),1].mean(),poses[320*ttt:320*(ttt+1),0].mean()])

ax.plot((poses[:,0]), label='HE', color='gray', linestyle='--', linewidth=3)
ax.plot(poses[:,1], label='VE', color='gray', linestyle='-', linewidth=3)

#ax.axvline(x=3000, color='red', linestyle='--')
#ax.axvline(x=3200, color='red', linestyle='--')

#ax.set_xlim(400,900)

ax.axis('off')
ax.grid(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

if saveFigs==1:
    fig2a.savefig(FIG_FOLDER+'/examp_eyepos.png', dpi=600, bbox_inches='tight');

In [ ]:
saveFigs = 1

dt = 50
data = io.loadmat('/Users/kendranoneman/Projects/mayo/NeuralDecoding/datasets/vars/vars-pa29dir4A-pre500-post300.mat')

spike_times  =  data['spike_times'] # spike times of all neurons
pos          =  data['pos'] # x and y eye positions
vel         =  data['vels']
acc          =  data['acc']
out_times    =  data['vels_times'] # times at which velocities were recorded
out_times    =  np.squeeze(out_times)

conditions = data['contConditions']

spike_times  =  np.squeeze(spike_times)
for i in range(spike_times.shape[0]):
    spike_times[i]  =  np.squeeze(spike_times[i])
    
unts = pd.read_csv('/Users/kendranoneman/Projects/mayo/NeuralDecoding/datasets/units/units-pa29dir4A-pre500-post300.csv')
fef = unts[(unts['BrainArea'] == 'FEF') & (unts['SNR'] > 4) & (unts['mnFR_bestDir'] < 30)].index
mt = unts[(unts['BrainArea'] == 'MT') & (unts['SNR'] > 1) & (unts['mnFR_bestDir'] < 30)].index

print(fef.shape[0])
print(mt.shape[0])
    
##############################################################################################
#trial = [1861] #1858. 1861. 1879. 1882. 1883.

trls = plot_trls[0:2]
#trls = [1858,1861]

spikes = []
for sublist in spike_times:
    s = sublist[np.logical_and(sublist>np.where(np.isin(conditions[:,0],trls))[0][0], sublist<np.where(np.isin(conditions[:,0],trls))[0][-1])]
    spikes.append(s)
    
fig, ax = plt.subplots(figsize=(15, 1.5))
fig.tight_layout()

ax.eventplot([spikes[i] for i in fef], colors='black', linelengths=0.5, linewidths=0.75, lineoffsets=np.arange(len([spikes[i] for i in fef])));
ax.eventplot([spikes[i] for i in mt], colors='black', linelengths=0.5, linewidths=0.75, lineoffsets=np.arange(len([spikes[i] for i in fef]),len([spikes[i] for i in fef])+len([spikes[i] for i in mt])));

ax.axis('off')
ax.grid(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

if saveFigs==1:
    fig.savefig(FIG_FOLDER+'/examp_spikes_crossVal.png', dpi=600, bbox_inches='tight')

##############################################################
fig2a, ax = plt.subplots(figsize=(15, 1.5))
fig2a.tight_layout()

trls = [plot_trls[0], plot_trls[1], plot_trls[2], plot_trls[3], 1861, 1858, 1879, 1882, 1883, 1861, 1862]

poses = pos[np.isin(conditions[:,0],trls)]
ttt = 4
print([poses[320*ttt:320*(ttt+1),1].mean(),poses[320*ttt:320*(ttt+1),0].mean()])

ax.plot((poses[:,0]), label='HE', color='gray', linestyle='--', linewidth=3)
ax.plot(poses[:,1], label='VE', color='gray', linestyle='-', linewidth=3)

#ax.set_xlim(400,900)

ax.axis('off')
ax.grid(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

if saveFigs==1:
    fig2a.savefig(FIG_FOLDER+'/examp_eyepos_crossVal.png', dpi=600, bbox_inches='tight');

## Figure 3. Eye kinematic reconstructions

In [ ]:
sub_folder = 'figures-2_4/'

column_names = ['sess','timePrePost','output_binWidth','downsample_factor','input_timeWindow','input_binWidth','model','output','nMT','nFEF','repeat','outer_fold','R2_train','rho_train','R2','rho','R2_shuf','rho_shuf','eval_full','prms','preProcess_time','train_time','test_time']
df0, conds0, yTest_all0, yTestPred_all0 = get_outputs_wTraces(DATA_FOLDER+sub_folder,["-o0-","r0001"],column_names) # position
df1, _, yTest_all1, yTestPred_all1 = get_outputs_wTraces(DATA_FOLDER+sub_folder,["-o1-","r0001"],column_names) # velocity
df2, _, yTest_all2, yTestPred_all2 = get_outputs_wTraces(DATA_FOLDER+sub_folder,["-o2-","r0001"],column_names) # acceleration

# Get trial numbers for "best" trial in each direction, will be used for all eye traces plots
plot_trls = get_exTrialLabels(df0,conds0,0,50,yTest_all0,yTestPred_all0,5)
print(plot_trls)

df0.head(2)

In [ ]:
fig, axes = plt.subplots(len(df0['model'].unique()),3,figsize=(10,12), sharex=True, sharey=False)
plt.rcParams.update({'font.size': 10.2})
fig.tight_layout()

lws = [2,2,1.7]

for i, m in enumerate(df0['model'].unique()):
    for c in range(3):
        if c==0:
            df = df0
            yt = yTest_all0
            yp = yTestPred_all0
        elif c==1:
            df = df1
            yt = yTest_all1
            yp = yTestPred_all1
        else:
            df = df2
            yt = yTest_all2
            yp = yTestPred_all2
            
        y_true, y_pred = get_eyeTraces(df,conds0,0,50,50,400,plot_trls,yt,yp,m)
        x = np.linspace(0,4*1.6,y_true.shape[0])

        axes[i,c].plot(x,y_true[:,0],linewidth=lws[c],linestyle='solid',color=(0.6,0.6,0.6))
        axes[i,c].plot(x,y_true[:,1],linewidth=lws[c],linestyle='dashed',color=(0.6,0.6,0.6))

        if m==4:
            axes[i,c].plot(x,y_pred[:,0]*np.std(y_true[:,0]),linewidth=lws[c],linestyle='solid',color=cols[i])
            axes[i,c].plot(x,y_pred[:,1]*np.std(y_true[:,1]),linewidth=lws[c],linestyle='dashed',color=cols[i])

            r2 = get_R2(y_true,y_pred * np.std(y_true,axis=0))

        else:
            axes[i,c].plot(x,y_pred[:,0],linewidth=lws[c],linestyle='solid',color=cols[i])
            axes[i,c].plot(x,y_pred[:,1],linewidth=lws[c],linestyle='dashed',color=cols[i])

            r2 = get_R2(y_true,y_pred)
            
        if c==0:
            axes[i,c].set_ylim(-18,18)
        elif c==1:
            axes[i,c].set_ylim(-45,45)
        else:
            axes[i,c].set_ylim(-270,270)

        text_box = f"$R^2_x$ = {r2[0]:.2f}, $R^2_y$ = {r2[1]:.2f}"
        axes[i,c].text(0.5, 0.96, text_box, transform=axes[i,c].transAxes, fontsize=9.69, ha='center', va='bottom', color=cols[i])

        axes[i,c].spines['right'].set_color('none')
        #axes[i].spines['bottom'].set_color('none')
        axes[i,c].spines['top'].set_color('none')
        
        if m!=8:
            axes[i,c].set_yticklabels([])
            
    
plt.subplots_adjust(wspace=0.153, hspace=0.25)

fig.savefig(FIG_FOLDER+'/allTraces_8models_r2xy.png', dpi=600, bbox_inches='tight')


## Figure 4. Summary decoder results

In [ ]:
sub_folder = 'figure8/'

column_names = ['sess','timePrePost','output_binWidth','downsample_factor','input_timeWindow','input_binWidth','model','output','nMT','nFEF','repeat','outer_fold','R2_train','rho_train','R2','rho','R2_shuf','rho_shuf','eval_full','prms','preProcess_time','train_time','test_time']
df0 = get_outputs(DATA_FOLDER+sub_folder,["-o0-","r0001"],column_names) # position
df1 = get_outputs(DATA_FOLDER+sub_folder,["-o1-","r0001"],column_names) # velocity
df2 = get_outputs(DATA_FOLDER+sub_folder,["-o2-","r0001"],column_names) # acceleration

df0.head(2)

In [ ]:
fig, axes = plt.subplots(3,1,figsize=(5,10), sharex=True) #6,5.5
plt.rcParams.update({'font.size': 12})
fig.tight_layout()

sessions = get_goodSessions(df0,0.2)
print(sessions)
for r in range(3):
    if r==0:
        df = df0
    elif r==1:
        df = df1
    else:
        df = df2

    for i,s in enumerate(sessions):  
        ##### standard R2 #####
        for q in range(2):
            if q==0:
                wf,wc,xgb,svr,dnn,rnn,gru,lstm = get_dfVal_folds(df.loc[(df['sess']==s)],50,50,400,'R2_mn')
            else:
                wf,wc,xgb,svr,dnn,rnn,gru,lstm = get_dfVal_folds(df.loc[(df['sess']==s)],50,50,400,'R2sh_mn')
        
            n=10
            means=([np.mean(wf),np.mean(wc),np.mean(xgb),np.mean(svr),np.mean(dnn),np.mean(rnn),np.mean(gru),np.mean(lstm)])    
            err=([np.std(wf)*np.sqrt(1./n+1./(n-1)),np.std(wc)*np.sqrt(1./n+1./(n-1)),np.std(xgb)*np.sqrt(1./n+1./(n-1)),np.std(svr)*np.sqrt(1./n+1./(n-1)),np.std(dnn)*np.sqrt(1./n+1./(n-1)),np.std(rnn)*np.sqrt(1./n+1./(n-1)),np.std(gru)*np.sqrt(1./n+1./(n-1)),np.std(lstm)*np.sqrt(1./n+1./(n-1))])

            ind = np.arange(len(err)) #X values for plotting
            if q==0:
                axes[r].plot(ind,means,color='gray',alpha=1,linestyle='solid',linewidth=0.75,label=s,zorder=0)
            else:
                axes[r].plot(ind,means,color='gray',alpha=0.25,linestyle='solid',linewidth=0.75,label=s,zorder=0)
                
            if s==29:
                n=10
                means=([np.mean(wf),np.mean(wc),np.mean(xgb),np.mean(svr),np.mean(dnn),np.mean(rnn),np.mean(gru),np.mean(lstm)])
                err=([np.std(wf)*np.sqrt(1./n+1./(n-1)),np.std(wc)*np.sqrt(1./n+1./(n-1)),np.std(xgb)*np.sqrt(1./n+1./(n-1)),np.std(svr)*np.sqrt(1./n+1./(n-1)),np.std(dnn)*np.sqrt(1./n+1./(n-1)),np.std(rnn)*np.sqrt(1./n+1./(n-1)),np.std(gru)*np.sqrt(1./n+1./(n-1)),np.std(lstm)*np.sqrt(1./n+1./(n-1))])

                ind = np.arange(len(err)) #X values for plotting

                if q==0:
                    axes[r].plot(ind,means,color='black',alpha=1,linestyle='solid',linewidth=2,label=s,zorder=2)
                    thiscol = cols
                    zz = 3
                else:
                    axes[r].plot(ind,means,color='black',alpha=0.4,linestyle='solid',linewidth=2,label=s,zorder=1)
                    thiscol = grays
                    zz = 2
                    
                for pos, y, yerr, color in zip(ind, means, err, thiscol): #Loop through methods and plot error bars

                    axes[r].errorbar(pos, y, yerr, lw=2.5, capsize=7.5, capthick=2, color=color,zorder=zz)

                    axes[r].set_xticks([0,1,2,3,4,5,6,7])
                    axes[r].set_xticklabels(['wf','wc','xgb','svr','dnn','rnn','gru','lstm'], rotation=0)

                    #Remove right and top boundaries, and make ticks inward
                    axes[r].tick_params(direction='in',bottom=0)
                    axes[r].spines['right'].set_color('none')
                    axes[r].spines['top'].set_color('none')

                    #Plot individual R2 values for each fold as an 'x'
                    scatter_x=np.reshape(np.transpose(np.ones((10,1))*range(8)),(80,1)) #Get x values for plotting (first 10 will have an x value of 0, second 10 will have an x value of 1, etc)
                    scatter_y=np.concatenate((wf,wc,xgb,svr,dnn,rnn,gru,lstm),axis=0) #Y values for plotting
                    colors_list=[] #Create a list of the colors that should be used when plotting each 'x'
                    for i in scatter_x.astype(int).reshape((1,-1))[0]:
                        colors_list.append(thiscol[i])

                    axes[r].scatter(scatter_x,scatter_y,c=colors_list,marker='.',s=70,alpha=0.05,zorder=2,edgecolors='face')

            if r==0:
                # Create a gap where the break is
                #axes[r].set_ylim(-1.1, 1)
                axes[r].spines['top'].set_visible(False)
                #axes[r].spines['left'].set_bounds(-1, 1)
                axes[r].spines['right'].set_visible(False)
            #if r==2:
            #    axes[r].set_ylim([-0.1,0.1])
                
axes[0].set_ylim([-1.15,1.15])
axes[0].set_yticks([-1,-0.5,0,0.5,1])

axes[1].set_ylim([-0.9,0.9])
axes[1].set_yticks([-0.8,-0.4,0,0.4,0.8])

axes[2].set_ylim([-0.13,0.13])
axes[2].set_yticks([-0.1,-0.05,0,0.05,0.1])

plt.subplots_adjust(wspace=0.15, hspace=0.065)

#fig.savefig(FIG_FOLDER+'/r2_allSessions.png', dpi=600, bbox_inches='tight')



In [ ]:
mods = [0,1,3,4,5,6,7,8]
outs = ['position','velocity','acceleration']

sessions = get_goodSessions(df0,0.2)
tot_count = 0
ns_count = 0
for s in sessions:
    for j,m in enumerate(mods):
        for i,o in enumerate(outs):
            if i==0:
                df = df0
            elif i==1:
                df = df1
            else:
                df = df2

            am = get_dfVal_folds(df.loc[(df['sess']==s)],50,50,400,'R2_mn')
            am_shuf = get_dfVal_folds(df.loc[(df['sess']==s)],50,50,400,'R2sh_mn')

            # Perform the Mann-Whitney U test
            statistic, p_value = mannwhitneyu(am[j], am_shuf[j], alternative='two-sided')
            # Output the test result
            
           # print(f"P-value: {p_value}")
            tot_count += 1
            if 1.00e-02 < p_value <= 5.00e-02:
                #print('*')
                blah = 1
            elif 1.00e-03 < p_value <= 1.00e-02:
                #print('**')
                blah = 2
            elif 1.00e-04 < p_value <= 1.00e-03:
                #print('***')
                blah = 4
            elif p_value <= 1.00e-04:
                #print('****')
                blah = 3
            else:
                print('ns')
                print('{} - {} - {}'.format(s,m,o))
                ns_count += 1
                print('\n')

## Figure 5. Output bin width

In [ ]:
sub_folder = 'figures-2_4/'

column_names = ['sess','timePrePost','output_binWidth','downsample_factor','input_timeWindow','input_binWidth','model','output','nMT','nFEF','repeat','outer_fold','R2_train','rho_train','R2','rho','R2_shuf','rho_shuf','eval_full','prms','preProcess_time','train_time','test_time']
df0, conds0, yTest_all0, yTestPred_all0 = get_outputs_wTraces(DATA_FOLDER+sub_folder,["-o0-","r0001"],column_names) # position
df1, _, yTest_all1, yTestPred_all1 = get_outputs_wTraces(DATA_FOLDER+sub_folder,["-o1-","r0001"],column_names) # velocity
df2, _, yTest_all2, yTestPred_all2 = get_outputs_wTraces(DATA_FOLDER+sub_folder,["-o2-","r0001"],column_names) # acceleration

# Get trial numbers for "best" trial in each direction, will be used for all eye traces plots
plot_trls = get_exTrialLabels(df0,conds0,0,50,yTest_all0,yTestPred_all0,5)
print(plot_trls)

df = pd.concat([df0,df1,df2],ignore_index=False)

df.head(2)

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 20})
fig, ax = plt.subplots(1, 3, figsize=(18, 6), sharex=True, sharey=False)#, gridspec_kw={'height_ratios': [3, 2]})

num_bins = 10
behs = ['R2_mn'] #, 'train_time'
kins = ['position', 'velocity', 'acceleration']

for k, kin in enumerate(kins):
    DF = df.loc[(df['output'] == kin) & (df['R2_mn'] >=0)]

    for i, beh in enumerate(behs):
        dff = DF[['model', 'output_binWidth', beh, 'outer_fold']]
        obw = dff['output_binWidth'].unique()

        df_mn = dff.groupby(['model', pd.cut(dff['output_binWidth'], bins=num_bins)])[beh].mean().reset_index(name="mR2")
        df_sd = dff.groupby(['model', pd.cut(dff['output_binWidth'], bins=num_bins)])[beh].std().reset_index(name="sR2")
        
        for i,m in enumerate(DF['model'].unique()):
            mod_mn = df_mn[(df_mn['model'] == m)]['mR2'].values
            mod_sd = (df_sd[(df_sd['model'] == m)]['sR2'].values) * np.sqrt(1. / 10 + 1. / (10 - 1))
            
            #statistic, p_value = f_oneway(mod_mn)
            # Check the p-value to determine significance
            '''
            alpha = 0.05
            if p_value < alpha:
                print("***")
                print(p_value)
            else:
                print("ns")
            '''

            # Get the centers of the bins for x-values
            bin_centers = [bin.mid for bin in df_mn[(df_mn['model'] == m)]['output_binWidth']]
            bin_mins = [bin.left for bin in df_mn[(df_mn['model'] == m)]['output_binWidth']]
            bin_maxs = [bin.right for bin in df_mn[(df_mn['model'] == m)]['output_binWidth']]
            
            # Round bin min and max values to the nearest integer
            bin_mins = [round(val) for val in bin_mins]
            bin_maxs = [round(val) for val in bin_maxs]

            ax[k].errorbar(bin_mins, mod_mn, mod_sd, lw=3, capsize=4, marker='o', alpha=0.9, markersize=7, capthick=2, color=cols[i], zorder=2)
            ax[k].errorbar(bin_mins, mod_mn, mod_sd, lw=3, capsize=4, marker='o', alpha=0.9, markersize=7, capthick=2, color=cols[i], zorder=3)

            # Set the x-tick labels as bin minimum and maximum for each point
            #x_tick_labels = [f"[{min_val}-{max_val})" for min_val, max_val in zip(bin_mins, bin_maxs)]
            #ax[k].set_xticks(bin_mins)
            #ax[k].set_xticklabels(bin_mins, rotation=45)
            ax[k].set_xticks([10,50,100,150,200,250,300])
            
        ax[k].spines['right'].set_color('none')
        ax[k].spines['top'].set_color('none')
    ax[0].set_ylim(0.33,0.92)
    ax[1].set_ylim(0.33,0.92)
    ax[2].set_ylim(-0.15,0.51)
    ax[0].set_yticks([0.4,0.5,0.6,0.7,0.8,0.9])
    ax[1].set_yticks([0.4,0.5,0.6,0.7,0.8,0.9])
    ax[2].set_yticks([-0.1,0,0.1,0.2,0.3])
    
#plt.subplots_adjust(wspace=0.25, hspace=0.5)

if saveFigs==1:
    fig.savefig(FIG_FOLDER+'/dto_8models_r2.png', dpi=600, bbox_inches='tight')

In [ ]:
# stats
from statsmodels.stats.anova import AnovaRM

num_bins = 15
kins = ['position', 'velocity', 'acceleration']

for k, kin in enumerate(kins):
    for i,m in enumerate(df['model'].unique()):
        DF = df.loc[(df['output'] == kin) & (df['model'] == m) & (df['output_binWidth']!=0) & (df['R2_mn'] >-1)]
        df_mn = DF.groupby([pd.cut(dff['output_binWidth'], bins=num_bins), 'outer_fold'])['R2_mn'].mean().reset_index()
        
        anova_model = AnovaRM(df_mn, 'R2_mn', 'outer_fold', within=['output_binWidth'])
        
        results = anova_model.fit()
        print('{}-{}'.format(kin,m))
        print(results)
        
        min_max_r2_values = df_mn.groupby('output_binWidth')['R2_mn'].agg(['min', 'max'])
        print(min_max_r2_values)

        # Extract the 'R2_mn' values for the smallest and largest 'output_binWidth'
        min_r2 = min_max_r2_values.loc[min_max_r2_values['R2_mn'].idxmin()]['R2_mn']
        max_r2 = min_max_r2_values.loc[min_max_r2_values['R2_mn'].idxmax()]['R2_mn']

        # Calculate the percent difference
        percent_difference = ((max_r2 - min_r2) / min_r2) * 100

        print(average_percent_difference)

        #df_sd = dff.groupby(['model', pd.cut(dff['output_binWidth'], bins=num_bins)])[beh].std().reset_index(name="sR2")
        

In [ ]:
bws = [10,50,100,300]
Nts = 6
lws = [2,2,1.7]

plt.rcParams.update({'font.size': 20})
fig, axes = plt.subplots(len(bws),3,figsize=(18,18), sharex=True, sharey=False)#,gridspec_kw={'height_ratios': [2,2,0.3,2,2,0.3,2,2]})
fig.tight_layout()

for b, bw in enumerate(bws):
    for c in range(3):
        if c==0:
            df = df0
            yt = yTest_all0
            yp = yTestPred_all0
        elif c==1:
            df = df1
            yt = yTest_all1
            yp = yTestPred_all1
        else:
            df = df2
            yt = yTest_all2
            yp = yTestPred_all2
            
        for i, m in enumerate(df0['model'].unique()):
            y_true, y_pred = get_eyeTraces(df,conds0,0,bw,50,400,plot_trls,yt,yp,m)
            
            x = np.linspace(0,4*1.6,y_true.shape[0])

            axes[b,c].plot(x,y_true[:,0],linewidth=lws[c],linestyle='solid',color=(0.6,0.6,0.6),zorder=0)

            if m==4:
                axes[b,c].plot(x,y_pred[:,0]*np.std(y_true[:,0]),linewidth=lws[c],linestyle='solid',color=cols[i],zorder=1)
                r2 = get_R2(y_true,y_pred * np.std(y_true[:,0]))

            else:
                axes[b,c].plot(x,y_pred[:,0],linewidth=lws[c],linestyle='solid',color=cols[i],zorder=1)
                r2 = get_R2(y_true,y_pred)
            
            if m==8:
                text_box = f"$R^2_x$ = {r2[0]:.2f}"
                axes[b,c].text(1, 0.02, text_box, transform=axes[b,c].transAxes, fontsize=18, ha='right', va='bottom', color=cols[i])

        axes[b,c].spines['right'].set_color('none')
        #axes[i].spines['bottom'].set_color('none')
        axes[b,c].spines['top'].set_color('none')
        axes[b,c].set_yticks([round(min(y_true[:,0])),0,round(max(y_true[:,0]))])

        #if m!=8:
            #axes[b,c].set_yticklabels([])
            
axes[0,0].set_xticks([0,2,4,6])
            
plt.subplots_adjust(wspace=0.25, hspace=0.05)
fig.savefig(FIG_FOLDER+'/dto_8traces_r2x.png', dpi=600, bbox_inches='tight')
            

## Figure 6. Input bin width and number of input bins/output bin

In [ ]:
sub_folder = 'figure5/'

column_names = ['sess','timePrePost','output_binWidth','downsample_factor','input_timeWindow','input_binWidth','model','output','nMT','nFEF','repeat','outer_fold','R2_train','rho_train','R2','rho','R2_shuf','rho_shuf','eval_full','prms','preProcess_time','train_time','test_time']
df0, conds0, _, _ = get_outputs_wTraces(DATA_FOLDER+sub_folder,["-o0-"],column_names)
df1, _, _, _ = get_outputs_wTraces(DATA_FOLDER+sub_folder,["-o1-"],column_names)
df2, _, _, _ = get_outputs_wTraces(DATA_FOLDER+sub_folder,["-o2-"],column_names)

df0.head(1)

In [ ]:
import matplotlib.ticker as ticker
import matplotlib.patches as patches
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from matplotlib.colors import TwoSlopeNorm

# Define the desired number of bins in x and y axes
desired_x_bins = 8 #9
desired_y_bins =8 #7
mdls = ['wf','wc','kf','xgb','svr','dnn','rnn','gru','lstm']

#################
plt.rcParams.update({'font.size': 14})
fig, ax = plt.subplots(8,3, figsize=(11,21), sharex=True, sharey=True)
fig.tight_layout()

for col in range(3):
    if col==0:
        df = df0
    elif col==1:
        df = df1
    else:
        df = df2
        
    for i,m in enumerate(df['model'].unique()):
        DF = df.loc[(df['model']==m) & (df['R2_mn'] >= 0) & (df['input_timeWindow']>10)]
        dff = DF[['R2_mn','input_binWidth','input_timeWindow']]
        
        df_m = dff.groupby([
            pd.qcut(dff['input_binWidth'], q=desired_x_bins, duplicates='drop'),
            pd.qcut(dff['input_timeWindow'], q=desired_y_bins, duplicates='drop')
        ])["R2_mn"].mean().reset_index(name="R2")

        # Pivot the data for heatmap
        result = df_m.pivot(index='input_timeWindow', columns='input_binWidth', values='R2')
        
        R2_values = result.values.astype(float)
        # Normalize the values using Min-Max scaler
        scaler = MinMaxScaler()
        normalized_values = scaler.fit_transform(R2_values)
        result2 = result.copy()
        result2.loc[:, :] = normalized_values
        
        x_labels = [f'{int(col.left) + 1}' for col in result.columns]
        y_labels = [f'{int(index.left) + 1}' for index in result.index]

        a = sns.heatmap(ax=ax[i,col], data=result2, cmap=pals[i], mask=result2.isnull(), alpha=0.99,
                        robust=True, xticklabels=x_labels, yticklabels=y_labels, center=0.45,
                        cbar_kws={'location': 'right', 'ticks': [0, 1],
                                'orientation': 'vertical', 'pad': 0.01, "shrink": 0.75,
                                'format': ticker.FuncFormatter(lambda x, pos: f'{x:.0f}')})
        
        #

        # Add custom annotations and outline for minimum and maximum values
        for y in range(result.shape[0]):
            for x in range(result.shape[1]-1):
                value = result.iloc[y, x]
                if value == np.min(result.iloc[:, :-1]):
                    a.text(x+0.5, y+0.5, f'{value:.2f}'.lstrip('0'), ha='center', va='center', color='black', fontsize=13)
                    a.add_patch(patches.Rectangle((x, y), 1, 1, edgecolor=cols[i], facecolor='none', lw=1))
                elif value == np.max(result):
                    a.text(x+0.5, y+0.5, f'{value:.2f}'.lstrip('0'), ha='center', va='center', color='black', fontsize=13)
                    a.add_patch(patches.Rectangle((x, y), 1, 1, edgecolor='black', facecolor='none', lw=1))
                else:
                    blh = 0
                    #a.text(x + 0.5, y + 0.5, f'{value:.2f}', ha='center', va='center', color='white')


        #, vmin=0.42, vmax=0.91,
        a.invert_yaxis()
        a.set_xticklabels(a.get_xticklabels(), rotation=45, ha="center")

        a.set(xlabel=None)
        a.set(ylabel=None)
        
        
plt.subplots_adjust(wspace=0.1, hspace=0.075)
fig.savefig(FIG_FOLDER+'/dti_heatmaps.png', dpi=600, bbox_inches='tight')
        

## Figure 7. Number of neurons

In [ ]:
sub_folder = 'figure6/'

column_names = ['sess','timePrePost','output_binWidth','downsample_factor','input_timeWindow','input_binWidth','model','output','nMT','nFEF','repeat','outer_fold','R2_train','rho_train','R2','rho','R2_shuf','rho_shuf','eval_full','prms','preProcess_time','train_time','test_time','neuron_inds']
df0, conds0, _, _ = get_outputs_wTraces(DATA_FOLDER+sub_folder,["-o0-"],column_names)
df1, _, _, _ = get_outputs_wTraces(DATA_FOLDER+sub_folder,["-o1-"],column_names)
df2, _, _, _ = get_outputs_wTraces(DATA_FOLDER+sub_folder,["-o2-"],column_names)

df0.head(1)

In [ ]:
plt.rcParams.update({'font.size': 22})
fig, ax = plt.subplots(1, 3, figsize=(27, 7), sharex=True, sharey=False)#, gridspec_kw={'height_ratios': [3, 2]})

#plt.subplots_adjust(hspace=0.1)

num_bins = 13
kins = ['position','velocity','acceleration']
behs = ['R2_mn']

for k, kin in enumerate(kins):
    if k==0:
        DF = df0
    elif k==1:
        DF = df1
    else:
        DF = df2
        
    DF = DF.drop('nFEF',axis=1)
    DF = DF.rename(columns={'nMT': 'numNeurons'})
    
    DF = DF[DF['R2_mn'] >= 0]

    for i, beh in enumerate(behs):
        dff = DF[['model', 'numNeurons', beh, 'outer_fold','repeat']]
        dff = dff.groupby(['model', 'numNeurons', 'outer_fold'])['R2_mn'].mean().reset_index()
        obw = dff['numNeurons'].unique()

        if num_bins==13:
            df_mn = dff.groupby(['model', 'numNeurons'])[beh].mean().reset_index(name="mR2")
            df_sd = dff.groupby(['model', 'numNeurons'])[beh].std().reset_index(name="sR2")
        else:
            df_mn = dff.groupby(['model', pd.cut(dff['numNeurons'], bins=num_bins)])[beh].mean().reset_index(name="mR2")
            df_sd = dff.groupby(['model', pd.cut(dff['numNeurons'], bins=num_bins)])[beh].std().reset_index(name="sR2")
          
        for i,m in enumerate(DF['model'].unique()):
            mod_mn = df_mn[(df_mn['model'] == m)]['mR2'].values
            mod_sd = (df_sd[(df_sd['model'] == m)]['sR2'].values) * np.sqrt(1. / 10 + 1. / (10 - 1))

            if num_bins==13:
                ax[k].errorbar(obw, mod_mn, mod_sd, lw=3, capsize=4, marker='o', alpha=0.9, markersize=7, capthick=2, color=cols[i], zorder=2)
                ax[k].errorbar(obw, mod_mn, mod_sd, lw=3, capsize=4, marker='o', alpha=0.9, markersize=7, capthick=2, color=cols[i], zorder=3)
                
                # Set the x-tick labels as bin minimum and maximum for each point
                x_tick_labels = obw
                ax[k].set_xticks(obw)
                ax[k].set_xticklabels(x_tick_labels, rotation=0)
            else:    
                # Get the centers of the bins for x-values
                bin_centers = [bin.mid for bin in df_mn[(df_mn['model'] == m)]['numNeurons']]
                bin_mins = [bin.left for bin in df_mn[(df_mn['model'] == m)]['numNeurons']]
                bin_maxs = [bin.right for bin in df_mn[(df_mn['model'] == m)]['numNeurons']]

                # Round bin min and max values to the nearest integer
                bin_mins = [round(val) for val in bin_mins]
                bin_maxs = [round(val) for val in bin_maxs]

                ax[k].errorbar(bin_centers, mod_mn, mod_sd, lw=3, capsize=4, marker='o', alpha=0.9, markersize=7, capthick=2, color=cols[i], zorder=2)
                ax[k].errorbar(bin_centers, mod_mn, mod_sd, lw=3, capsize=4, marker='o', alpha=0.9, markersize=7, capthick=2, color=cols[i], zorder=3)

                # Set the x-tick labels as bin minimum and maximum for each point
                x_tick_labels = [f"[{min_val}-{max_val})" for min_val, max_val in zip(bin_mins, bin_maxs)]
                #x_tick_labels = bin_mins
                ax[k].set_xticks(bin_centers)
                ax[k].set_xticklabels(x_tick_labels, rotation=45)
            
        ax[k].spines['right'].set_color('none')
        ax[k].spines['top'].set_color('none')
    
plt.subplots_adjust(wspace=0.15, hspace=0.1)
ax[2].set_xticks([5,15,25,35,45,55,65])

ax[0].set_yticks([0,0.2,0.4,0.6,0.8])
ax[0].set_ylim(0.09,0.9)
ax[1].set_yticks([0,0.2,0.4,0.6,0.8])
ax[1].set_ylim(0.09,0.9)
ax[2].set_yticks([0,0.05,0.1])
ax[2].set_ylim(-0.02,0.25)

fig.savefig(FIG_FOLDER+'/nneurons_r2.png', dpi=600, bbox_inches='tight')


## Figure 8. Evaluation metrics

In [ ]:
pd.options.mode.chained_assignment = None  # 'warn' or None

sub_folder = 'figures-2_4/' # observations sweep
column_names = ['sess','timePrePost','output_binWidth','downsample_factor','input_timeWindow','input_binWidth','model','output','nMT','nFEF','repeat','outer_fold','R2_train','rho_train','R2','rho','R2_shuf','rho_shuf','eval_full','prms','preProcess_time','train_time','test_time']
df0, conds0, _, _ = get_outputs_wTraces(DATA_FOLDER+sub_folder,["s29-","r0001"],column_names)
df0 = calc_evalMetrics(df0,sess_size,0)
df0.head(1)

sub_folder = 'figure5/' # features sweep
column_names = ['sess','timePrePost','output_binWidth','downsample_factor','input_timeWindow','input_binWidth','model','output','nMT','nFEF','repeat','outer_fold','R2_train','rho_train','R2','rho','R2_shuf','rho_shuf','eval_full','prms','preProcess_time','train_time','test_time']
df1, conds1, _, _ = get_outputs_wTraces(DATA_FOLDER+sub_folder,["s29-"],column_names)
df1 = calc_evalMetrics(df1,sess_size,0)
df1.head(1)

sub_folder = 'figure6/' # neurons sweep
column_names = ['sess','timePrePost','output_binWidth','downsample_factor','input_timeWindow','input_binWidth','model','output','nMT','nFEF','repeat','outer_fold','R2_train','rho_train','R2','rho','R2_shuf','rho_shuf','eval_full','prms','preProcess_time','train_time','test_time','neuron_inds']
df2, conds2, _, _ = get_outputs_wTraces(DATA_FOLDER+sub_folder,["s29-"],column_names)
df2 = calc_evalMetrics(df2,sess_size,0)
df2.head(1)

In [ ]:
df2 = df2.drop('nFEF',axis=1)
df2 = df2.rename(columns={'nMT': 'numNeurons'})

In [ ]:
np.arange(10,300,40)

In [ ]:
from matplotlib.ticker import ScalarFormatter

num_bins = 12
behs = ['train_time','inference_time','robust_diff']
varbls = ['output_binWidth','num_feat','numNeurons']
kins = ['position','velocity']
lns = ['-','--','-.']
alphs = [1,0.6,0.3]

plt.rcParams.update({'font.size': 20})
fig, ax = plt.subplots(len(behs), len(varbls), figsize=(14, 22), sharex='col', sharey='row')#, gridspec_kw={'height_ratios': [3, 2]})
fig.tight_layout()

for v, vr in enumerate(varbls):
    if v==0:
        ddf = df0
    elif v==1:
        ddf = df1.loc[df1['input_binWidth']<=20]
    else:
        ddf = df2
        
    for n, kin in enumerate(kins):
        df = ddf.loc[ddf['output']==kin]

        for k, beh in enumerate(behs):
            dff = df[['model', vr, beh, 'outer_fold']]
            obw = dff[vr].unique()

            df_mn = dff.groupby(['model', pd.cut(dff[vr], bins=num_bins)])[beh].mean().reset_index(name="mR2")
            df_sd = dff.groupby(['model', pd.cut(dff[vr], bins=num_bins)])[beh].std().reset_index(name="sR2")

            for i,m in enumerate(df['model'].unique()):
                mod_mn = df_mn[(df_mn['model'] == m)]['mR2'].values
                mod_sd = (df_sd[(df_sd['model'] == m)]['sR2'].values) * np.sqrt(1. / 10 + 1. / (10 - 1))

                # Get the centers of the bins for x-values
                bin_centers = [bin.mid for bin in df_mn[(df_mn['model'] == m)][vr]]
                bin_mins = [bin.left for bin in df_mn[(df_mn['model'] == m)][vr]]
                bin_maxs = [bin.right for bin in df_mn[(df_mn['model'] == m)][vr]]

                # Round bin min and max values to the nearest integer
                bin_mins = [round(val) for val in bin_mins]
                bin_maxs = [round(val) for val in bin_maxs]

                ax[k,v].errorbar(bin_centers, mod_mn, mod_sd, lw=3, capsize=4, linestyle=lns[n], marker='o', alpha=alphs[n], markersize=7, capthick=2, color=cols[i], zorder=2)
                #ax[k,v].errorbar(bin_centers, mod_mn, mod_sd, lw=3, capsize=4, linestyle=lns[n], marker='o', alpha=0.9, markersize=7, capthick=2, color=cols[i], zorder=3)

                x_tick_labels = [int(center) for center in bin_centers]
                x_tick_labels = [int(center) for i, center in enumerate(bin_centers)]

                #if v==0:
                #    x_tick_labels = [round((sess_size/x)/10000,1) for x in x_tick_labels]
                #    ax[k,v].invert_xaxis()

                ax[k, v].set_xticks(bin_centers)
                ax[k, v].set_xticklabels(x_tick_labels, rotation=45)

            ax[k,v].spines['right'].set_color('none')
            ax[k,v].spines['top'].set_color('none')
            
    #if v==0:
    #    ax[k,v].invert_xaxis()
    
    #ax[0,0].set_xticks(np.arange(20,320,60))
#ax[0,0].set_ylim([-40,450])
#ax[1,0].set_ylim([-0.6,8])

plt.subplots_adjust(wspace=0.25, hspace=0.05)
#fig.savefig(FIG_FOLDER+'/evalMetrics.png', dpi=600, bbox_inches='tight')

In [ ]:
from matplotlib.ticker import ScalarFormatter

num_bins = 10
behs = ['robust_diff']
varbls = ['output_binWidth','num_feat','numNeurons']
kins = ['position','velocity','acceleration']
lns = ['-','--','-.']
alphs = [0.9,0.6,0.3]

plt.rcParams.update({'font.size': 20})
fig, ax = plt.subplots(len(kins), len(varbls), figsize=(14, 12), sharex='col', sharey=True)#, gridspec_kw={'height_ratios': [3, 2]})
fig.tight_layout()

for v, vr in enumerate(varbls):
    if v==0:
        ddf = df0
    elif v==1:
        ddf = df1.loc[df['input_binWidth']==20]
    else:
        ddf = df2
        
    for n, kin in enumerate(kins):
        df = ddf.loc[ddf['output']==kin]

        for k, beh in enumerate(behs):
            dff = df[['model', vr, beh, 'outer_fold']]
            obw = dff[vr].unique()

            df_mn = dff.groupby(['model', pd.cut(dff[vr], bins=num_bins)])[beh].mean().reset_index(name="mR2")
            df_sd = dff.groupby(['model', pd.cut(dff[vr], bins=num_bins)])[beh].std().reset_index(name="sR2")

            for i,m in enumerate(df['model'].unique()):
                mod_mn = df_mn[(df_mn['model'] == m)]['mR2'].values
                mod_sd = (df_sd[(df_sd['model'] == m)]['sR2'].values) * np.sqrt(1. / 10 + 1. / (10 - 1))

                # Get the centers of the bins for x-values
                bin_centers = [bin.mid for bin in df_mn[(df_mn['model'] == m)][vr]]
                bin_mins = [bin.left for bin in df_mn[(df_mn['model'] == m)][vr]]
                bin_maxs = [bin.right for bin in df_mn[(df_mn['model'] == m)][vr]]

                # Round bin min and max values to the nearest integer
                bin_mins = [round(val) for val in bin_mins]
                bin_maxs = [round(val) for val in bin_maxs]

                ax[n,v].errorbar(bin_centers, mod_mn, mod_sd, lw=3, capsize=4, linestyle=lns[n], marker='o', alpha=alphs[n], markersize=7, capthick=2, color=cols[i], zorder=2)
                #ax[k,v].errorbar(bin_centers, mod_mn, mod_sd, lw=3, capsize=4, linestyle=lns[n], marker='o', alpha=0.9, markersize=7, capthick=2, color=cols[i], zorder=3)

                x_tick_labels = [int(center) for center in bin_centers]
                x_tick_labels = [int(center) for i, center in enumerate(bin_centers)]

                #if v==0:
                #    x_tick_labels = [round((sess_size/x)/10000,1) for x in x_tick_labels]
                #    ax[k,v].invert_xaxis()

                ax[n, v].set_xticks(bin_centers)
                ax[n, v].set_xticklabels(x_tick_labels, rotation=45)

            ax[n,v].spines['right'].set_color('none')
            ax[n,v].spines['top'].set_color('none')
            
    #if v==0:
    #    ax[k,v].invert_xaxis()
    
    #ax[0,0].set_xticks(np.arange(20,320,60))
#ax[0,0].set_ylim([-40,450])
#ax[1,0].set_ylim([-0.6,8])

plt.subplots_adjust(wspace=0.25, hspace=0.05)
#fig.savefig(FIG_FOLDER+'/evalMetrics.png', dpi=600, bbox_inches='tight')